In [1]:
import pandas as pd
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec, KeyedVectors
from gensim import downloader
import gzip 
import math
import itertools
from time import time
from tqdm import tqdm
import tqdm.notebook as tq
import os
import pandas as pd
import json
import gensim.downloader as api

# Create iterative embedding evaluator

In [2]:
class eval_embeddings():
    def __init__(self, dir, files, dl=False):
        self.fils = files
        self.dl = dl
        #Change directory
        os.chdir(dir)

    def load_wv(self, fil):
        if self.dl == False:
            if os.path.isfile(fil):
                model = gensim.models.Word2Vec.load(fil)
                return model.wv
            else: 
                print(f"{fil} embeddings do not exist!")
                return 0
        else: 
            model = api.load(fil)
            return model
        
    def iterate(self, fils=None):
        self.evals = {}
        for fil in self.fils:
            wv = self.load_wv(fil)
            if wv == 0: #load_wv returns wv if embeddings don't exist
                continue
            else:
                self.evals[fil] = {} #create dic for decade
                self.eval_embed(wv, fil)
        self.evals_df = pd.DataFrame(self.evals)
               
    def eval_embed(self, wv, fil):
        print(f"Starting evaluation for {fil}")
        # Evaluate one set of embeddings (for one decade) - scores stored under "analogy_score", and "word_pair_pearson/spearman"
        analog = wv.evaluate_word_analogies('C:/ProgramData/Anaconda3/Lib/site-packages/gensim/test/test_data/questions-words.txt')
        word_pairs = wv.evaluate_word_pairs('C:/ProgramData/Anaconda3/Lib/site-packages/gensim/test/test_data/wordsim353.tsv')
        self.evals[fil]["analogy_score"] = analog[0]
        self.evals[fil]["word_pair_pearson"]= word_pairs[0] #tuple with correlation score & 2-tailed p-value
        self.evals[fil]["word_pair_spearman"]= word_pairs[1]#tuple with correlation score & 2-tailed p-value
        print(f"{fil} embeddings evaluated.")
            
    

# Evaluate all decades

In [3]:
#get filenames
year_files = ['vectors-'+str(1900+*i)+'-ngram' for i in range(100)]
#run evaluator
evaluator = eval_embeddings(dir = "D:/google_ngrams/vectors/vectors_per_year", files = year_files, dl=False)
#evaluator.iterate()

SyntaxError: invalid syntax (Temp/ipykernel_6280/3362059211.py, line 2)

In [ ]:
#get filenames
year_files = ['w2vmodel_ng5_'+str(1900+10*i)+'_full' for i in range(10)]
#run evaluator
evaluator = eval_embeddings(dir = "D:/google_ngrams/vectors/gb_12_vectors_1", files = decade_files, dl=False)
#evaluator.iterate()

# Information on other baselines

In [ ]:
info = api.info()
for corpus_name, corpus_data in sorted(info['models'].items()):
    print(
        '%s (%d records): %s' % (
            corpus_name,
            corpus_data.get('num_records', -1),
            corpus_data['description'][:10] + '...',
        )
    )
corps = [corpus_name for corpus_name, corpus_data in sorted(info['models'].items())]
print(corps)

# Evaluate other baselines

In [4]:
external_models = ['conceptnet-numberbatch-17-06-300', 'fasttext-wiki-news-subwords-300', 'glove-twitter-100', 'glove-twitter-200', 'glove-twitter-25', 'glove-twitter-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-wiki-gigaword-50', 'word2vec-google-news-300'] #'__testing_word2vec-matrix-synopsis',  'word2vec-ruscorpora-300' 
       

external_eval = eval_embeddings(dir = "D:/google_ngrams/vectors/gb_12_vectors_1", files=external_models, dl=True)
external_eval.iterate()

Starting evaluation for conceptnet-numberbatch-17-06-300


ValueError: x and y must have length at least 2.

In [ ]:
evaluator.evals

# Single case (old)

In [ ]:
#testing single case

#load word vectors from model
print(os.listdir())
year = "-cl"
fil = f'w2vmodel_ng5_{year}_full'
model = gensim.models.Word2Vec.load(fil)
wv = model.wv
#evaluate
evals = {}
evals[year] = {}
analog = model.wv.evaluate_word_analogies('C:/ProgramData/Anaconda3/Lib/site-packages/gensim/test/test_data/questions-words.txt')
word_pairs = model.wv.evaluate_word_pairs('C:/ProgramData/Anaconda3/Lib/site-packages/gensim/test/test_data/wordsim353.tsv')
evals[year]["analogy_score"] = analog[0]
for section in analog[1]:
    try:
        evals[f"analogy_{section['section']}"] = 100*len(section["correct"])/(len(section["incorrect"])+len(section["correct"]))
    except Exception as e:
        print("Error: ", e)
evals[year]["word_pair_pearson"]=word_pairs[0] #tuple with correlation score & 2-tailed p-value
evals[year]["word_pair_spearman"]=word_pairs[1]#tuple with correlation score & 2-tailed p-value
print(f"{fil} embeddings evaluated.")
evals